In [2]:
import numpy as np
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
import nltk
import re
import spacy
nltk.download('stopwords')
nltk.download('punkt')
nlp = spacy.load('fr_core_news_sm')

/home/lahad/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
[nltk_data] Downloading package stopwords to /home/lahad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/lahad/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Séparation l'ensemble d'entraînement en 2 parties: entrainement et validation

In [3]:
# Charger les données
data = pd.read_csv("../data/train.csv")

train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

train_data.to_csv("../data/train_split.csv", index=False)
val_data.to_csv("../data/validation_split.csv", index=False)


## Chargement des données

In [4]:
# Charger les données
train_data = pd.read_csv("../data/train_split.csv")
val_data = pd.read_csv("../data/validation_split.csv")
test_data = pd.read_csv("../data/test.csv")

train_data.head()

"- 1 pâte brisée - 150 g de raisins bien mûrs - 3 oeufs - 100 g de sucre - 3 cuillères à soupe de crème fraîche - 60 g de poudre d'amandes"

## Statistiques sur les données

In [5]:
# Nombre de documents dans chaque ensemble
num_train_docs = len(train_data)
num_val_docs = len(val_data)
num_test_docs = len(test_data)

print("Nombre de documents dans chaque ensemble :")
print("Entraînement :", num_train_docs)
print("Validation :", num_val_docs)
print("Test :", num_test_docs)

# Nombre de documents dans chaque ensemble pour chaque classe
train_label_counts = train_data['type'].value_counts()
val_label_counts = val_data['type'].value_counts()
test_label_counts = test_data['type'].value_counts()

print("\nNombre de documents par classe dans chaque ensemble :")
print("Ensemble d'entraînement :")
print(train_label_counts)
print("\nEnsemble de validation :")
print(val_label_counts)
print("\nEnsemble de test :")
print(test_label_counts)

# Pourcentage de documents dans chaque ensemble pour chaque classe
train_label_percentages = (train_data['type'].value_counts(normalize=True) * 100).round(2)
val_label_percentages = (val_data['type'].value_counts(normalize=True) * 100).round(2)
test_label_percentages = (test_data['type'].value_counts(normalize=True) * 100).round(2)

print("\nPourcentage de documents par classe dans chaque ensemble :")
print("Ensemble d'entraînement :")
print(train_label_percentages)
print("\nEnsemble de validation :")
print(val_label_percentages)
print("\nEnsemble de test :")
print(test_label_percentages)


Nombre de documents dans chaque ensemble :
Entraînement : 9978
Validation : 2495
Test : 1388

Nombre de documents par classe dans chaque ensemble :
Ensemble d'entraînement :
type
Plat principal    4644
Dessert           3036
Entrée            2298
Name: count, dtype: int64

Ensemble de validation :
type
Plat principal    1158
Dessert            726
Entrée             611
Name: count, dtype: int64

Ensemble de test :
type
Plat principal    644
Dessert           407
Entrée            337
Name: count, dtype: int64

Pourcentage de documents par classe dans chaque ensemble :
Ensemble d'entraînement :
type
Plat principal    46.54
Dessert           30.43
Entrée            23.03
Name: proportion, dtype: float64

Ensemble de validation :
type
Plat principal    46.41
Dessert           29.10
Entrée            24.49
Name: proportion, dtype: float64

Ensemble de test :
type
Plat principal    46.40
Dessert           29.32
Entrée            24.28
Name: proportion, dtype: float64


### Separer les données en attributs et étiquettes


In [6]:
# Prétraitement des données d'entraînement
train_data['text'] = train_data['titre'] + " " + train_data['recette']
train_data = train_data[['text', 'type']]

# Prétraitement des données de validation
val_data['text'] = val_data['titre'] + " " + val_data['recette']
val_data = val_data[['text', 'type']]

# Prétraitement des données de test
test_data['text'] = test_data['titre'] + " " + test_data['recette']
test_data = test_data[['text', 'type']]

# Separer les attributs et les étiquettes
X_train = train_data['text']
y_train = train_data['type']
X_val = val_data['text']
y_val = val_data['type']
X_test = test_data['text']
y_test = test_data['type']

###  Nettoyage (Normalisation) des donnees 

In [7]:

stopwords_fr = set(stopwords.words('french'))
stemmer = FrenchStemmer()

def clean_text(text):
    words = simple_preprocess(text, min_len=1)  # Tokenization et conversion en minuscules
    words = [word for word in words if word not in stopwords_fr]
    doc = nlp(" ".join(words))
    words = [token.lemma_ for token in doc] # Lemmatization
    words = [stemmer.stem(word) for word in words] # Stemming
    cleaned_text = ' '.join(words)

    return cleaned_text

# Appliquer le nettoyage sur les données 
X_train = X_train.apply(clean_text)
X_val = X_val.apply(clean_text)
X_test = X_test.apply(clean_text)



## Baseline

In [8]:

dummy = DummyClassifier(strategy='stratified', random_state=42)
dummy.fit(X_train, y_train)

# # Prédire sur les données de developpement
# y_pred_val = dummy.predict(X_val)
# # Afficher le rapport de classification pour les données de developpement
# print("Classification Report for Validation Data:")
# print(classification_report(y_val, y_pred_val))

# Prédire sur les données de test
y_pred_test = dummy.predict(X_test)
print("Classification Report for Test Data:")
print(classification_report(y_test, y_pred_test,digits=3))


Classification Report for Test Data:
                precision    recall  f1-score   support

       Dessert      0.309     0.319     0.314       407
        Entrée      0.266     0.252     0.259       337
Plat principal      0.493     0.495     0.494       644

      accuracy                          0.385      1388
     macro avg      0.356     0.356     0.356      1388
  weighted avg      0.384     0.385     0.384      1388



#### Run2: TF-IDF + SVM

In [9]:
# Vectorisation des textes
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
X_val_tfidf = tfidf.transform(X_val)
X_test_tfidf = tfidf.transform(X_test)

# Création du modèle SVM avec noyau gaussien (RBF)
svm_rbf = SVC(kernel='rbf', C=5, random_state=42)
svm_rbf.fit(X_train_tfidf, y_train)

# Prédiction sur les données de developpement
# y_pred_val = svm_rbf.predict(X_val_tfidf)
# # Evaluation sur les données de developpement
# accuracy_val = accuracy_score(y_val, y_pred_val)
# precision_val = precision_score(y_val, y_pred_val, average='weighted')
# recall_val = recall_score(y_val, y_pred_val, average='weighted')
# f1_val = f1_score(y_val, y_pred_val, average='weighted')
# conf_matrix_val = confusion_matrix(y_val, y_pred_val)
# class_report_val = classification_report(y_val, y_pred_val,digits=3)
# Affichage des résultats sur les données de developpement
# print("Evaluation Metrics sur les donnees de developpement:")
# print(f"Accuracy: {accuracy_val}")
# print(f"Precision: {precision_val}")
# print(f"Recall: {recall_val}")
# print(f"F1 Score: {f1_val}")
# print("Confusion Matrix:")
# print(conf_matrix_val)
# print("Classification Report:")
# print(class_report_val)

# Prédiction sur les données de test
y_pred_test = svm_rbf.predict(X_test_tfidf)
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test = precision_score(y_test, y_pred_test, average='weighted')
recall_test = recall_score(y_test, y_pred_test, average='weighted')
f1_test = f1_score(y_test, y_pred_test, average='weighted')
conf_matrix_test = confusion_matrix(y_test, y_pred_test)
class_report_test = classification_report(y_test, y_pred_test,digits=3)

# Affichage des résultats sur les données de test
print("\nEvaluation Metrics sur les donnees de Test:")
print(f"Accuracy: {accuracy_test}")
print(f"Precision: {precision_test}")
print(f"Recall: {recall_test}")
print(f"F1 Score: {f1_test}")
print("Confusion Matrix:")
print(conf_matrix_test)
print("Classification Report:")
print(class_report_test)



Evaluation Metrics sur les donnees de Test:
Accuracy: 0.8840057636887608
Precision: 0.8825647252273752
Recall: 0.8840057636887608
F1 Score: 0.883168505059553
Confusion Matrix:
[[405   1   1]
 [  3 253  81]
 [  6  69 569]]
Classification Report:
                precision    recall  f1-score   support

       Dessert      0.978     0.995     0.987       407
        Entrée      0.783     0.751     0.767       337
Plat principal      0.874     0.884     0.879       644

      accuracy                          0.884      1388
     macro avg      0.879     0.876     0.877      1388
  weighted avg      0.883     0.884     0.883      1388



### Run3: Word2Vec + SVM

In [10]:
# Modèle Word2Vec
corpus = [sentence.split() for sentence in X_train]
word2vec_model = Word2Vec(corpus, vector_size=150, window=10, min_count=2, workers=1, sg=1)

def document_vector(word2vec_model, doc):
    # Filtrer les mots absents dans le vocabulaire
    doc = [word for word in doc if word in word2vec_model.wv.key_to_index]
    if len(doc) != 0:
        return np.mean(word2vec_model.wv[doc], axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

# Préparation des données pour Word2Vec
X_train_word2vec = np.array([document_vector(word2vec_model, doc.split()) for doc in X_train])
X_val_word2vec = np.array([document_vector(word2vec_model, doc.split()) for doc in X_val])
X_test_word2vec = np.array([document_vector(word2vec_model, doc.split()) for doc in X_test])

# Modèle SVM
svm_classifier = SVC(kernel='rbf', C=5, random_state=42)
svm_classifier.fit(X_train_word2vec, y_train)

# Prédiction sur les données de developpement
# y_pred_val = svm_classifier.predict(X_val_word2vec)
# accuracy_val = accuracy_score(y_val, y_pred_val)
# precision_val = precision_score(y_val, y_pred_val, average='weighted')
# recall_val = recall_score(y_val, y_pred_val, average='weighted')
# f1_val = f1_score(y_val, y_pred_val, average='weighted')
# conf_matrix_val = confusion_matrix(y_val, y_pred_val)
# class_report_val = classification_report(y_val, y_pred_val,digits=4)
# Affichage des résultats
# print("Evaluation Metrics sur les donnees de  developpement :")
# print(f"Accuracy: {accuracy_val}")
# print(f"Precision: {precision_val}")
# print(f"Recall: {recall_val}")
# print(f"F1 Score: {f1_val}")
# print("Confusion Matrix:")
# print(conf_matrix_val)
# print("Classification Report:")
# print(class_report_val)


# Prédiction sur les données de test
y_pred_test = svm_classifier.predict(X_test_word2vec)
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test = precision_score(y_test, y_pred_test, average='weighted')
recall_test = recall_score(y_test, y_pred_test, average='weighted')
f1_test = f1_score(y_test, y_pred_test, average='weighted')
conf_matrix_test = confusion_matrix(y_test, y_pred_test)
class_report_test = classification_report(y_test, y_pred_test,digits=3)

print("\nEvaluation Metrics sur les donnees de Test:")
print(f"Accuracy: {accuracy_test}")
print(f"Precision: {precision_test}")
print(f"Recall: {recall_test}")
print(f"F1 Score: {f1_test}")
print("Confusion Matrix:")
print(conf_matrix_test)
print("Classification Report:")
print(class_report_test)


Evaluation Metrics sur les donnees de Test:
Accuracy: 0.8782420749279539
Precision: 0.876086808027787
Recall: 0.8782420749279539
F1 Score: 0.8753295923936499
Confusion Matrix:
[[404   1   2]
 [  5 229 103]
 [  4  54 586]]
Classification Report:
                precision    recall  f1-score   support

       Dessert      0.978     0.993     0.985       407
        Entrée      0.806     0.680     0.738       337
Plat principal      0.848     0.910     0.878       644

      accuracy                          0.878      1388
     macro avg      0.878     0.861     0.867      1388
  weighted avg      0.876     0.878     0.875      1388



### Run4: Bag of Words + SVM

In [11]:
# Vectorisation des textes
vectorizer = CountVectorizer()

# Transformation des données textuelles en vecteurs de compte
X_train_counts = vectorizer.fit_transform(X_train)
X_val_counts = vectorizer.transform(X_val)
X_test_counts = vectorizer.transform(X_test)

# Création et entraînement du modèle SVM
svm_classifier = SVC(kernel="rbf", C=5, random_state=42)
svm_classifier.fit(X_train_counts, y_train)

# Prédiction sur les données de test
y_pred_test = svm_classifier.predict(X_test_counts)

# Evaluation sur les données de test
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test = precision_score(y_test, y_pred_test, average='weighted')
recall_test = recall_score(y_test, y_pred_test, average='weighted')
f1_test = f1_score(y_test, y_pred_test, average='weighted')
conf_matrix_test = confusion_matrix(y_test, y_pred_test)
class_report_test = classification_report(y_test, y_pred_test,digits=3)


# # Prédiction sur les données de developpement
# y_pred_val = svm_classifier.predict(X_val_counts)
# # Evaluation sur les données de developpement
# accuracy_val = accuracy_score(y_val, y_pred_val)
# precision_val = precision_score(y_val, y_pred_val, average='weighted')
# recall_val = recall_score(y_val, y_pred_val, average='weighted')
# f1_val = f1_score(y_val, y_pred_val, average='macro')
# conf_matrix_val = confusion_matrix(y_val, y_pred_val)
# class_report_val = classification_report(y_val, y_pred_val,digits=4)
# # Affichage des résultats sur les données de developpement
# print("Evaluation Metrics sur les donnees de developpement :")
# print(f"Accuracy: {accuracy_val}")
# print(f"Precision: {precision_val}")
# print(f"Recall: {recall_val}")
# print(f"F1 Score: {f1_val}")
# print("Confusion Matrix:")
# print(conf_matrix_val)
# print("Classification Report:")
# print(class_report_val)

# Affichage des résultats sur les données de test
print("\nEvaluation Metrics sur les donnees de Tests:")
print(f"Accuracy: {accuracy_test}")
print(f"Precision: {precision_test}")
print(f"Recall: {recall_test}")
print(f"F1 Score: {f1_test}")
print("Confusion Matrix:")
print(conf_matrix_test)
print("Classification Report:")
print(class_report_test)


Evaluation Metrics sur les donnees de Tests:
Accuracy: 0.872478386167147
Precision: 0.870466324950637
Recall: 0.872478386167147
F1 Score: 0.8712973055054815
Confusion Matrix:
[[405   1   1]
 [  5 244  88]
 [  7  75 562]]
Classification Report:
                precision    recall  f1-score   support

       Dessert      0.971     0.995     0.983       407
        Entrée      0.762     0.724     0.743       337
Plat principal      0.863     0.873     0.868       644

      accuracy                          0.872      1388
     macro avg      0.866     0.864     0.865      1388
  weighted avg      0.870     0.872     0.871      1388

